In [1]:
import os
import cv2
import numpy as np
from collections import deque
import tensorflow as tf

from keras.models import load_model


In [ ]:
current_model=''

# Use LRCN model

In [3]:
model=load_model('LRCN_model__DateTime_2023_09_06__08_55_34__Loss_0.19016073644161224__Accuracy_0.935251772403717.h5')
current_model='LRCN'
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_3 (TimeDis  (None, 20, 64, 64, 16)   448       
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, 20, 16, 16, 16)   0         
 tributed)                                                       
                                                                 
 time_distributed_5 (TimeDis  (None, 20, 16, 16, 16)   0         
 tributed)                                                       
                                                                 
 time_distributed_6 (TimeDis  (None, 20, 16, 16, 32)   4640      
 tributed)                                                       
                                                                 
 time_distributed_7 (TimeDis  (None, 20, 4, 4, 32)    

# Use convLSTM model

In [9]:
model=load_model('convlstm_model__DateTime_2023_09_06__08_49_18__Loss_0.29193222522735596__Accuracy_0.9280575513839722.h5')
current_model='convLSTM'
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, 20, 62, 62, 4)     1024      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 20, 31, 31, 4)    0         
 )                                                               
                                                                 
 time_distributed (TimeDistr  (None, 20, 31, 31, 4)    0         
 ibuted)                                                         
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 20, 29, 29, 8)     3488      
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 20, 15, 15, 8)    0         
 3D)                                                             
                                                        

# Test

In [4]:
IMAGE_HEIGHT,IMAGE_WIDTH=64,64
SEQUENCE_LENGTH=20
DATASET_DIR='UCF50'
CLASS_LIST=['Punch',"JumpingJack","HighJump","JumpRope"]

In [10]:
test_vdo_dir='test_videos'
os.makedirs(test_vdo_dir,exist_ok=True)
video_title=f'testo_{current_model}.mp4'
input_video_file_path='testo.mp4'

In [11]:
def predict_on_video(video_file_path,out_file_path,SEQUENCE_LENGTH):
  video_reader=cv2.VideoCapture(video_file_path)
  frame_rate = video_reader.get(cv2.CAP_PROP_FPS)
  original_width=int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
  origial_height=int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
  video_writer=cv2.VideoWriter(out_file_path,cv2.VideoWriter_fourcc('M','P','4','V'),
                                video_reader.get(cv2.CAP_PROP_FPS),(original_width,origial_height))
  frames_queue=deque(maxlen=SEQUENCE_LENGTH)
  predicted_class_name=''

  while video_reader.isOpened():
    ok,frame=video_reader.read()

    if not ok:
      break

    resized_frame=cv2.resize(frame,(IMAGE_HEIGHT,IMAGE_WIDTH))
    normalized_frame=resized_frame/255

    frames_queue.append(normalized_frame)

    if len(frames_queue)==SEQUENCE_LENGTH:
        predicted_label_prob=model.predict(np.expand_dims(frames_queue,axis=0))[0]

        predicted_label=np.argmax(predicted_label_prob)

        predicted_class_name=CLASS_LIST[predicted_label]


        if max(predicted_label_prob)<=0.99:
          print(str(max(predicted_label_prob))+" Class: "+predicted_class_name+" was deleted!")
          predicted_class_name=''
        else:
          print(str(max(predicted_label_prob))+" Class: "+predicted_class_name+" was kept!")
    cv2.putText(frame,predicted_class_name+" FPS:"+str(frame_rate),(10,30),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)

    video_writer.write(frame)

  video_reader.release()
  video_writer.release()


In [12]:
output_video_file_path=f'{test_vdo_dir}/{video_title}-Output-Seqlen{SEQUENCE_LENGTH}.mp4'

predict_on_video(input_video_file_path,output_video_file_path,SEQUENCE_LENGTH)

1/1 [==============================] - 1s 1s/step
0.4258056 Class: JumpingJack was deleted!
1/1 [==============================] - 0s 175ms/step
0.46909797 Class: JumpingJack was deleted!
1/1 [==============================] - 0s 164ms/step
0.40859428 Class: HighJump was deleted!
1/1 [==============================] - 0s 166ms/step
0.66319066 Class: HighJump was deleted!
1/1 [==============================] - 0s 198ms/step
0.85763484 Class: HighJump was deleted!
1/1 [==============================] - 0s 190ms/step
0.92680097 Class: HighJump was deleted!
1/1 [==============================] - 0s 173ms/step
0.94454986 Class: HighJump was deleted!
1/1 [==============================] - 0s 178ms/step
0.96002847 Class: HighJump was deleted!
1/1 [==============================] - 0s 170ms/step
0.97198796 Class: HighJump was deleted!
1/1 [==============================] - 0s 177ms/step
0.97599447 Class: HighJump was deleted!
1/1 [==============================] - 0s 180ms/step
0.9821346 Class